In [58]:
import requests
import pandas as pd
from IPython.display import display

# City of Chicago Crime Data API Endpoint
API_URL = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"

# Define date range for filtering (2015 to today)
params = {
    "$limit": 5000,  # Increase limit to gather more data
    "$where": "date >= '2015-01-01T00:00:00' AND date < '2025-12-31T23:59:59'",  # Filter for 2015 - Today
    "$order": "date DESC"  # Get the most recent crimes first
}

# Fetch data from the API
response = requests.get(API_URL, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Convert data to a Pandas DataFrame
    df = pd.DataFrame(data)

    # Convert date column to datetime format (if available)
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # Display the DataFrame in Jupyter Notebook / VS Code
    display(df)

else:
    print(f"Error: Unable to fetch data (Status Code: {response.status_code})")


,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,13757563,JJ159806,2025-02-24 00:00:00,106XX S OGLESBY AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,10,51,14,1194026,1835043,2025,2025-03-03T15:41:36.000,41.702293529,-87.565139627,"{'latitude': '41.702293529', 'longitude': '-87..."
1,13763346,JJ167090,2025-02-24 00:00:00,088XX S WOOD ST,1170,DECEPTIVE PRACTICE,IMPERSONATION,RESIDENCE,False,False,...,18,71,11,NaN,NaN,2025,2025-03-03T15:41:36.000,NaN,NaN,NaN
2,13760970,JJ164037,2025-02-24 00:00:00,055XX S EMERALD AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,True,...,16,68,06,1172246,1868093,2025,2025-03-03T15:41:36.000,41.793492485,-87.64392184,"{'latitude': '41.793492485', 'longitude': '-87..."
3,13757812,JJ160305,2025-02-24 00:00:00,072XX S PAULINA ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,17,67,07,1166266,1856649,2025,2025-03-03T15:41:36.000,41.762218185,-87.666175568,"{'latitude': '41.762218185', 'longitude': '-87..."
4,13757547,JJ160082,2025-02-24 00:00:00,002XX W MONROE ST,2820,OTHER OFFENSE,TELEPHONE THREAT,RESTAURANT,False,False,...,42,32,08A,1174585,1899875,2025,2025-03-03T15:41:36.000,41.880653112,-87.634396649,"{'latitude': '41.880653112', 'longitude': '-87..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,13749084,JJ149952,2025-02-14 17:40:00,109XX S DOTY AVE W,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,...,9,50,06,1185440,1832668,2025,2025-02-22T16:03:07.000,41.695982095,-87.596653142,"{'latitude': '41.695982095', 'longitude': '-87..."
4996,13749184,JJ149876,2025-02-14 17:36:00,060XX S WABASH AVE,031A,ROBBERY,ARMED - HANDGUN,ALLEY,False,False,...,20,40,03,1177750,1865019,2025,2025-02-22T16:03:07.000,41.784934232,-87.623832282,"{'latitude': '41.784934232', 'longitude': '-87..."
4997,13749022,JJ149854,2025-02-14 17:30:00,009XX N LARAMIE AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,RESIDENCE - GARAGE,False,False,...,37,25,07,1141518,1905847,2025,2025-02-22T16:03:07.000,41.8977158,-87.755669081,"{'latitude': '41.8977158', 'longitude': '-87.7..."
4998,13749101,JJ149933,2025-02-14 17:30:00,039XX W LAWRENCE AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,...,33,14,14,1149058,1931603,2025,2025-02-22T16:03:07.000,41.968249849,-87.727306531,"{'latitude': '41.968249849', 'longitude': '-87..."


In [65]:
import pandas as pd 
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load the CSV file
file_path = "Shootings.csv"
df = pd.read_csv(file_path, parse_dates=['DATE'], low_memory=False)

# Extract year and week number
df['Year'] = df['DATE'].dt.year
df['Week_Number'] = df['DATE'].dt.isocalendar().week
df['Month'] = df['DATE'].dt.month

# Filter only years 2019-2024
df_filtered = df[df['Year'].between(2019, 2024)]

# Filter only HOMICIDE and NON-FATAL SHOOTINGS cases
df_violent = df_filtered[df_filtered['VICTIMIZATION_PRIMARY'].str.contains("HOMICIDE|NON-FATAL", na=False, case=False)]

# Define custom colors for each year
year_colors = {
    2019: "#A99A93",
    2020: "#7D726E",
    2021: "#524D49",
    2022: "#C0B5AF",
    2023: "#E0D7D2",
    2024: "#3A3633",
    2025: "#282522"
}

# Rolling average dropdown selector
rolling_avg_selector = widgets.Dropdown(
    options=[('7 Day rolling Average', 'weekly'), ('Monthly Average', 'monthly')],
    value='weekly',
    description="Rolling Avg:",
    style={'description_width': 'initial'}
)

# Create an output widget for updating the graph dynamically
output = widgets.Output()

def update_graph(rolling_window):
    with output:
        output.clear_output(wait=True)
        fig = go.Figure()
        
        if rolling_window == 'monthly':  # Monthly aggregation
            victimization_counts = df_violent.groupby(['Year', 'Month']).size().unstack(level=0)
            x_axis = list(range(1, 13))  # Months (1-12)
            x_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
            time_label = "Month"
        else:  # Weekly basis
            victimization_counts = df_violent.groupby(['Year', 'Week_Number']).size().unstack(level=0)
            x_axis = victimization_counts.index
            x_labels = x_axis  # Keep week numbers as labels
            time_label = "Week"

        for year in victimization_counts.columns:
            fig.add_trace(go.Scatter(
                x=x_axis, y=victimization_counts[year],
                mode='lines+markers',
                name=str(year),
                line=dict(color=year_colors.get(year, "#000000")),
                hoverinfo='x+y+name',  # Removes HTML tags in hover text
                hovertemplate=
                "<b>Rolling Average:</b> %{text}<br>"
                "<b>Year:</b> " + str(year) + "<br>"
                "<b>" + time_label + ":</b> %{x}<br>"
                "<b>Total Fatal and Non-Fatal Shootings:</b> %{y}",
                text=["7-day" if rolling_window == "weekly" else "Monthly"] * len(x_axis)
            ))
        
        fig.update_layout(
            title=f"Homicide & Non-Fatal Shootings Trends (2019-2024) - {rolling_window} View",
            xaxis_title=time_label,
            xaxis=dict(tickmode='array', tickvals=x_axis, ticktext=x_labels),
            yaxis_title="Number of Victimizations",
            template="plotly_white",
            hovermode="closest",  # Ensures only the hovered line shows hover info
            font=dict(family="Arial", size=14)
        )
        
        fig.show()

# Display widgets on the top left
ui = widgets.HBox([rolling_avg_selector])
widgets.interactive(update_graph, rolling_window=rolling_avg_selector)
display(ui, output)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_632\3592370841.py:8: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Output()

In [70]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load the CSV file
file_path = "Shootings.csv"
df = pd.read_csv(file_path, parse_dates=['DATE'], low_memory=False)

# Extract year and week number
df['Year'] = df['DATE'].dt.year
df['Week_Number'] = df['DATE'].dt.isocalendar().week
df['Month'] = df['DATE'].dt.month

# Filter only years 2019-2024
df_filtered = df[df['Year'].between(2019, 2024)]

# Filter only HOMICIDE and NON-FATAL SHOOTINGS cases
df_violent = df_filtered[df_filtered['VICTIMIZATION_PRIMARY'].str.contains("HOMICIDE|NON-FATAL", na=False, case=False)]

# Rolling average dropdown selector
rolling_avg_selector = widgets.Dropdown(
    options=[('7 Day rolling Average', 'weekly'), ('Monthly Average', 'monthly')],
    value='weekly',
    description="Rolling Avg:",
    style={'description_width': 'initial'}
)

# Create an output widget for updating the graph dynamically
output = widgets.Output()

def update_graph(rolling_window):
    with output:
        output.clear_output(wait=True)
        fig = go.Figure()
        
        if rolling_window == 'monthly':  # Monthly aggregation
            victimization_counts = df_violent.groupby(['Year', 'Month']).size().unstack(level=0)
            x_axis = list(range(1, 13))  # Months (1-12)
            x_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
            time_label = "Month"
        else:  # Weekly basis
            victimization_counts = df_violent.groupby(['Year', 'Week_Number']).size().unstack(level=0)
            x_axis = victimization_counts.index
            x_labels = x_axis  # Keep week numbers as labels
            time_label = "Week"
        
        for year in victimization_counts.columns:
            fig.add_trace(go.Scatter(
                x=x_axis, y=victimization_counts[year],
                mode='lines+markers',
                name=str(year),
                hoverinfo='x+y+name',
                hovertemplate=
                "<b>Year:</b> " + str(year) + "<br>"
                "<b>" + time_label + ":</b> %{x}<br>"
                "<b>Victimizations:</b> %{y}",
                text=[str(year)] * len(x_axis),  # Ensures only the hovered line shows its data
                line_shape='spline'  # Smooth line for better hovering
            ))
        
        fig.update_layout(
            title=f"Homicide & Non-Fatal Shootings Trends (2019-2024) - {rolling_window} View",
            xaxis_title=time_label,
            xaxis=dict(tickmode='array', tickvals=x_axis, ticktext=x_labels),
            yaxis_title="Number of Victimizations",
            template="plotly_white",
            hovermode="closest"  # Ensures only the closest hovered point shows hover info
        )
        
        fig.show()

# Display widgets on the top left
ui = widgets.HBox([rolling_avg_selector])
widgets.interactive(update_graph, rolling_window=rolling_avg_selector)
display(ui, output)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_632\560461937.py:8: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Output()

In [66]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load the CSV file
file_path = "Shootings.csv"
df = pd.read_csv(file_path, parse_dates=['DATE'], low_memory=False)

# Extract year and month
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month

# Filter only years 2019-2024
df_filtered = df[df['Year'].between(2019, 2024)]

# Filter only HOMICIDE and NON-FATAL SHOOTINGS cases
df_violent = df_filtered[df_filtered['VICTIMIZATION_PRIMARY'].str.contains("HOMICIDE|NON-FATAL", na=False, case=False)]

# Define regions based on wards
ward_to_region = {
    40: 'Far North Side', 48: 'Far North Side', 49: 'Far North Side', 50: 'Far North Side',
    32: 'North Side', 43: 'North Side', 44: 'North Side', 46: 'North Side',
    30: 'Northwest Side', 31: 'Northwest Side', 33: 'Northwest Side', 35: 'Northwest Side',
    36: 'Northwest Side', 38: 'Northwest Side', 39: 'Northwest Side', 41: 'Northwest Side', 45: 'Northwest Side',
    24: 'West Side', 27: 'West Side', 28: 'West Side', 37: 'West Side',
    2: 'Central', 3: 'Central', 4: 'Central', 25: 'Central', 42: 'Central',
    12: 'Southwest Side', 14: 'Southwest Side', 15: 'Southwest Side', 18: 'Southwest Side', 22: 'Southwest Side', 23: 'Southwest Side',
    5: 'South Side', 6: 'South Side', 7: 'South Side', 8: 'South Side', 9: 'South Side', 10: 'South Side', 20: 'South Side', 21: 'South Side',
    13: 'Far Southwest Side', 16: 'Far Southwest Side', 17: 'Far Southwest Side', 19: 'Far Southwest Side',
    34: 'Far Southeast Side', 35: 'Far Southeast Side', 36: 'Far Southeast Side', 41: 'Far Southeast Side'
}

# Add a new column for region
df_violent['Region'] = df_violent['WARD'].map(ward_to_region)

# Handle missing or invalid values in the 'Region' column
df_violent['Region'] = df_violent['Region'].fillna('Unknown')  # Replace NaN with 'Unknown'

# Year dropdown selector
year_selector = widgets.Dropdown(
    options=sorted(df_violent['Year'].unique()),  # Sort years
    value=2021,  # Default to 2021
    description="Year:",
    style={'description_width': 'initial'}
)

# Create an output widget for updating the graph dynamically
output = widgets.Output()

def update_graph(year):
    with output:
        output.clear_output(wait=True)
        fig = go.Figure()
        
        # Filter data by selected year
        df_year = df_violent[df_violent['Year'] == year]
        
        # Group by Region and Month, then count the number of incidents
        monthly_counts = df_year.groupby(['Region', 'Month']).size().unstack(level=0)
        
        # Months (1-12) and their labels
        x_axis = list(range(1, 13))
        x_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
        
        # Add a trace for each region
        for region in monthly_counts.columns:
            fig.add_trace(go.Scatter(
                x=x_axis, y=monthly_counts[region],
                mode='lines+markers',
                name=region,
                hoverinfo='x+y+name',
                hovertemplate="<b>Region:</b> " + region + "<br><b>Month:</b> %{x}<br><b>Victimizations:</b> %{y}",
                line_shape='spline'  # Smooth line for better visualization
            ))
        
        # Update layout
        fig.update_layout(
            title=f"Homicide & Non-Fatal Shootings Trends ({year}) - Monthly View",
            xaxis_title="Month",
            xaxis=dict(tickmode='array', tickvals=x_axis, ticktext=x_labels),
            yaxis_title="Number of Victimizations",
            template="plotly_white",
            hovermode="closest"  # Ensures only the hovered line shows hover info
        )
        
        fig.show()

# Display widgets on the top left
ui = widgets.HBox([year_selector])
widgets.interactive(update_graph, year=year_selector)
display(ui, output)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_632\1199966127.py:8: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\Kevin\AppData\Local\Temp\ipykernel_632\1199966127.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Kevin\AppData\Local\Temp\ipykernel_632\1199966127.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Output()